In [ ]:
#Mount Google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install seqeval
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 22.0 MB/s 
     |████████████████████████████████| 596 kB 69.4 MB/s 
     |████████████████████████████████| 86 kB 7.4 MB/s 
     |████████████████████████████████| 6.6 MB 58.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 2.4 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=8fbcc5591d1075af5ac370d4eacf1a2207508ae3761f4b151594bffa3f1a9673
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-whee

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification, BertTokenizer, BertForSequenceClassification
from seqeval.metrics import classification_report
import pickle
import nltk
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from nltk import word_tokenize
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
import emoji
def give_emoji_free_text(text):
    return emoji.replace_emoji(text, replace='')

In [ ]:
model_name = "dccuchile/bert-base-spanish-wwm-cased"
tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=True)

Downloading:   0%|          | 0.00/236k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/134 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/364 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/648 [00:00<?, ?B/s]

In [ ]:
all_tags_1 = sorted(set(['Not Crime', 'Crime']), reverse=True) #['Abduction', 'Arms Possession And Trafficking', 'Crime Against Animals', 'Drug Trafficking', 'Escape', 'False Report', 'Falsification and Illegal Sale', 'Fraud', 'Homicide', 'Human Trafficking', 'Injury', 'Phishing', 'Public Disorder', 'Robbery', 'Sexual Violence', 'Terrorism', 'Theft']
num_tags_1 = len(all_tags_1)
print("Number of tags:", num_tags_1)
print("Tags:", all_tags_1)

Number of tags: 2
Tags: ['Not Crime', 'Crime']


In [ ]:
tag2index_1 = {tag: i for i, tag in enumerate(all_tags_1)}
index2tag_1 = {i: tag for tag, i in tag2index_1.items()}

In [ ]:
index2tag_1

{0: 'Not Crime', 1: 'Crime'}

In [ ]:
model_dir = Path('/content/drive/Shareddrives/CrimeTrack: Tracking Crime on the Web/Model/Model Crime Not Crime Classification_97_2')
max_seq_len, all_tags, tag2index, index2tag = pickle.load((model_dir / 'parrameters.pk').open('rb'))

model_crime = BertForSequenceClassification.from_pretrained(model_dir, num_labels=len(tag2index_1)) # load bert pre-trained model 

model_crime.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
def inference(sentence, tokenizer):
    inputs = tokenizer(sentence,padding='max_length', max_length = 128, truncation=True, return_tensors="pt")

    # move to gpu
    ids = inputs["input_ids"].to(device)
    mask = inputs["attention_mask"].to(device)
    # forward pass
    outputs = model_crime(ids, attention_mask=mask)
    logits = outputs[0]

    active_logits = logits.view(-1, model_crime.num_labels) # shape (batch_size * seq_len, num_labels)
    flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

    tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
    
    token_predictions = [index2tag_1[i] for i in flattened_predictions.cpu().numpy()]

    return token_predictions

In [ ]:
path_1 = '/content/drive/Shareddrives/CrimeTrack: Tracking Crime on the Web/Data/Dataset/Tweets Guardia Civil.xlsx'
path_2 = '/content/drive/Shareddrives/CrimeTrack: Tracking Crime on the Web/Data/Dataset/Tweets Policia.xlsx'

guardia = pd.read_excel(path_1)
policia = pd.read_excel(path_2)

In [ ]:
guardia['crime_label'] = ''
for i in range(len(guardia)):
  sentence = give_emoji_free_text(guardia.tweet[i])
  entities = inference(sentence,tokenizer)
  guardia['crime_label'][i] = entities[0] 
  if (i) % 1000 == 0:
        print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000


In [ ]:
policia['crime_label'] = ''
for i in range(len(policia)):
  sentence = give_emoji_free_text(policia.tweet[i])
  entities = inference(sentence,tokenizer)
  policia['crime_label'][i] = entities[0] 
  if (i) % 1000 == 0:
        print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000


In [ ]:
all_tags_1 = sorted(set(['Abduction', 'Arms Possession And Trafficking', 'Crime Against Animals', 'Drug Trafficking', 'Escape', 'False Report', 'Falsification and Illegal Sale', 'Fraud', 'Homicide', 'Human Trafficking', 'Injury', 'Phishing', 'Public Disorder', 'Robbery', 'Sexual Violence', 'Terrorism', 'Theft'])) #
num_tags_1 = len(all_tags_1)
print("Number of tags:", num_tags_1)
print("Tags:", all_tags_1)

Number of tags: 17
Tags: ['Abduction', 'Arms Possession And Trafficking', 'Crime Against Animals', 'Drug Trafficking', 'Escape', 'False Report', 'Falsification and Illegal Sale', 'Fraud', 'Homicide', 'Human Trafficking', 'Injury', 'Phishing', 'Public Disorder', 'Robbery', 'Sexual Violence', 'Terrorism', 'Theft']


In [ ]:
tag2index_1 = {tag: i for i, tag in enumerate(all_tags_1)}
index2tag_1 = {i: tag for tag, i in tag2index_1.items()}

In [ ]:
tag2index_1

{'Abduction': 0,
 'Arms Possession And Trafficking': 1,
 'Crime Against Animals': 2,
 'Drug Trafficking': 3,
 'Escape': 4,
 'False Report': 5,
 'Falsification and Illegal Sale': 6,
 'Fraud': 7,
 'Homicide': 8,
 'Human Trafficking': 9,
 'Injury': 10,
 'Phishing': 11,
 'Public Disorder': 12,
 'Robbery': 13,
 'Sexual Violence': 14,
 'Terrorism': 15,
 'Theft': 16}

In [ ]:
model_dir = Path('/content/drive/Shareddrives/CrimeTrack: Tracking Crime on the Web/Model/Model Crime Type_90_TOP')
max_seq_len, all_tags, tag2index, index2tag = pickle.load((model_dir / 'parrameters.pk').open('rb'))

model_crime = BertForSequenceClassification.from_pretrained(model_dir, num_labels=len(tag2index_1)) # load bert pre-trained model 

model_crime.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
guardia['crime_type'] = ''
for i in guardia.loc[guardia.crime_label == 'Crime'].index:
  sentence = give_emoji_free_text(guardia.loc[guardia.crime_label == 'Crime'].tweet[i])
  entities = inference(sentence,tokenizer)
  guardia['crime_type'][i] = entities[0] 
  if (i) % 1000 == 0:
        print(i)

0
2000
4000
63000


In [ ]:
policia['crime_type'] = ''
for i in policia.loc[policia.crime_label == 'Crime'].index:
  sentence = give_emoji_free_text(policia.loc[policia.crime_label == 'Crime'].tweet[i])
  entities = inference(sentence,tokenizer)
  policia['crime_type'][i] = entities[0] 
  if (i) % 1000 == 0:
        print(i)

13000
33000
35000
38000
39000
41000


In [ ]:
policia.to_excel('/content/drive/Shareddrives/CrimeTrack: Tracking Crime on the Web/Data/Dataset/Tweets Policia (Classified).xlsx', index=False)
guardia.to_excel('/content/drive/Shareddrives/CrimeTrack: Tracking Crime on the Web/Data/Dataset/Tweets Guardia Civil (Classified).xlsx', index=False)